In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Data Cleaning**

In [ ]:
import pandas as pd
import seaborn as sns

data = pd.read_csv('/kaggle/input/clothing-dataset-full/images.csv')
data.loc[data['label']=='Not sure','label'] = 'Not_sure'
#data['image'] = data['image'] + '.jpg'

labels=data['label']
features=data.drop('label',1)

labels_values=labels.value_counts()
print(labels_values)

# data['label_cat'] = data['label'] + ' ' + data['kids'].astype(str)
# proc_data=data[['image', 'label_cat']]
# print(type(proc_data))

p= sns.countplot(data=data['image'], x=data['label'])

In [ ]:
labels_y=data.label
labels_num1=labels_y[:100]
labels_num1

In [ ]:
labels_unique=labels_num1.unique()
print(labels_unique.shape)
# data.loc[(data.label) & (data.image == '4285fab0-751a-4b74-8e9b-43af05deee22')]

In [ ]:
# foods = list(os.walk('../input/food41/images/'))[0][1]
nr_foods = 17
labels_values=list(labels_values)

idx_to_name = {i:x for (i,x) in enumerate(labels_unique[:nr_foods])}
name_to_idx = {x:i for (i,x) in enumerate(labels_unique[:nr_foods])}

idx_to_name

In [ ]:
import cv2
from PIL import Image
from tqdm import tqdm 

data_l = []
img_size = (128, 128)

for img in tqdm(data['image'][:100]):
    imgs = [cv2.resize(cv2.imread(f'/kaggle/input/clothing-dataset-full/images_original/{img}.jpg'), img_size, interpolation=cv2.INTER_AREA)]
    for i in imgs:
        data_l.append(i)


In [ ]:
labels_y=[]

import numpy as np
for i in labels_num1:
    labels_y.append(name_to_idx[i])
len(labels_y)

In [ ]:
labels_y_s=pd.Series(labels_y)
type(labels_y_s)

In [ ]:
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import OneHotEncoder


enc = OneHotEncoder()
enc.fit(labels_y_s.values.reshape(-1, 1))

y_transformed = enc.transform(labels_y_s.values.reshape(-1, 1))

import scipy
y_one_hot=scipy.sparse.csr_matrix.toarray(y_transformed)

y_one_hot.shape

In [ ]:

data_l_1 = np.array(data_l)
data_l_2 = data_l_1 / 255
data_l_3 = data_l_2.astype('float32')
data_l_3.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_l_3, y_one_hot,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    shuffle=True)

In [ ]:
def f1(y_true, y_pred): 
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [ ]:
# f1 as a loss function (worse results)

def macro_soft_f1(y, y_hat):

    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1
    macro_cost = tf.reduce_mean(cost)
    
    return macro_cost

In [ ]:
def results_plot(history, title):
    
    fig, ax = plt.subplots(2,1)
    ax[0].plot(history.history['loss'], color='b', label="train loss")
    ax[0].plot(history.history['val_loss'], color='r', label="val loss",axes=ax[0])
    legend = ax[0].legend()

    ax[1].plot(history.history['accuracy'], color='b', label="train acc")
    ax[1].plot(history.history['val_accuracy'], color='r',label="val acc")
    
    legend = ax[1].legend()
    ax[0].set_title(title)
    
    plt.plot()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import Sequential, Input, initializers, optimizers, callbacks
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, Flatten, ReLU, PReLU
from tensorflow.keras.metrics import AUC
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
seed=47

In [ ]:
shape = (128, 128, 3)
num_targets = y_one_hot.shape[1]

metrics = ['accuracy', f1]
fit_params = {
        'x': X_train,
        'y': y_train,
        'validation_data': (X_test, y_test),
        'verbose': 1}

In [ ]:

def base_model(epochs, lr):

    model = Sequential()
    model.add(Input(shape))
    model.add(Flatten())
    model.add(Dense(num_targets, activation='softmax',
           kernel_initializer=initializers.RandomNormal(0, 1, seed=seed)))
    
    model.compile(
        loss=CategoricalCrossentropy(),
        optimizer=optimizers.SGD(learning_rate=lr),
        metrics=metrics)
    
    history = model.fit(**fit_params,
                        epochs=epochs)
    del model
#     gc.collect()
    
    return history

In [ ]:
base_model_history = base_model(epochs=100, lr=0.01)

In [ ]:
def tanh_model(epochs, lr):

    model = Sequential()
    model.add(Input(shape))
    model.add(Flatten())
    model.add(Dense(256,
                    activation='tanh',
                    kernel_initializer=initializers.GlorotNormal()))
    model.add(Dense(64,
                    activation='tanh',
                    kernel_initializer=initializers.GlorotNormal()))
    model.add(Dense(num_targets,
                    activation='softmax',
                    kernel_initializer=initializers.RandomNormal(0, 1, seed=seed)))
    
    model.compile(
        loss=CategoricalCrossentropy(),
        optimizer=optimizers.SGD(learning_rate=lr),
        metrics=metrics)
    
    history = model.fit(**fit_params,
                        epochs=epochs,
                             )
    del model
#     gc.collect()
    
    return history

In [ ]:
tanh_model_history = tanh_model(epochs=100, lr=0.01)

In [ ]:
def relu_model(epochs, lr):

    model = Sequential()
    model.add(Input(shape))
    model.add(Flatten())
    model.add(Dense(256, activation='relu',
           kernel_initializer=initializers.HeNormal()))
    model.add(Dense(64, activation='relu',
           kernel_initializer=initializers.HeNormal()))
    model.add(Dense(num_targets, activation='softmax',
           kernel_initializer=initializers.RandomNormal(0, 1, seed=seed)))
    
    model.compile(
        loss=CategoricalCrossentropy(),
        optimizer=optimizers.SGD(learning_rate=lr),
        metrics=metrics)
    
    history = model.fit(**fit_params,
                        epochs=epochs,
                             )
    del model

    
    return history

In [ ]:
relu_model_history = relu_model(epochs=100, lr=0.01)

In [ ]:
results_plot(base_model_history, 'base')
results_plot(tanh_model_history, 'tanh')
results_plot(relu_model_history, 'relu')

Найкращі результати показали tanh та relu. relu має кращий показник accuracy, і на валідаційному, і на тренувальному сеті.Проте модель tanh показала досить гарний, в порівнянні з іншими моделями,значення f1, що є дуже важливим для датасету з несбалансованими класами. Перенавчання в жодній моделі не спостерігається, можливе тільки недонавчання у базовій моделі.

In [ ]:
def tune_sgd(lr_list):
    
    results = pd.DataFrame()
    
    for lr in lr_list:
        relu_model_history = tanh_model(epochs=10, lr=lr)
        results[f'lr {lr}'] = pd.Series({k:v[-1] for k, v in relu_model_history.history.items()})
    
    return results

In [ ]:
results_tune_sgd = tune_sgd([0.1, 0.05, 0.001])

In [ ]:
results_tune_sgd

In [ ]:
import time

def compare_activations(activations_list):
    
    results = pd.DataFrame()
    for act in activations_list:
        model = Sequential()
        model.add(Input(shape))
        model.add(Flatten())
        model.add(Dense(256, activation=list(act.values())[0]))
        model.add(Dense(64, activation='tanh'))
        model.add(Dense(num_targets, activation='softmax',
               kernel_initializer=initializers.RandomNormal(0, 1, seed=seed)))

        model.compile(
            loss=CategoricalCrossentropy(),
            optimizer=optimizers.SGD(learning_rate=0.001),
            metrics=metrics)
        
        start = time.time()
        history = model.fit(**fit_params,
                            epochs=10)
        end = time.time()
        
        del model
#         gc.collect()
        
        results[f'{list(act)[0]}'] = pd.concat([pd.Series({'time': end - start}), pd.Series({k:v[-1] for k, v in history.history.items()})])
        
    return results

In [ ]:
activations_list = [
    {'prelu': tf.keras.layers.PReLU()},
    {'leaky_relu': tf.keras.layers.LeakyReLU()}, 
    {'elu': tf.keras.layers.ELU()}
                   ]

results_compare_activations = compare_activations(activations_list)

In [ ]:
results_compare_activations

In [ ]:
def compare_optimizers(optimizers_list):
    
    results = pd.DataFrame()
    for opt in optimizers_list:
        
        model = Sequential()
        model.add(Input(shape))
        model.add(Flatten())
        model.add(Dense(256,
                        activation='tanh',
                        kernel_initializer=initializers.GlorotNormal()))
        model.add(Dense(64,
                        activation='tanh',
                        kernel_initializer=initializers.GlorotNormal()))
        model.add(Dense(num_targets,
                        activation='softmax',
                        kernel_initializer=initializers.RandomNormal(0, 1, seed=seed)))

        model.compile(
            loss=CategoricalCrossentropy(),
            optimizer=list(opt.values())[0],
            metrics=metrics)
        
        start = time.time()
        history = model.fit(**fit_params,
                            epochs=10)
        end = time.time()
        
        del model

        
        results[f'{list(opt)[0]}'] = pd.concat([pd.Series({'time': end - start}), pd.Series({k:v[-1] for k, v in history.history.items()})])
        
    return results

In [ ]:
results_compare_optimizers = compare_optimizers([{'Nesterov': optimizers.SGD(nesterov=True)},
                                                 {'Adagrad': optimizers.Adagrad()},
                                                 {'Adadelta': optimizers.Adadelta()},
                                                 {'Adam': optimizers.Adam()}])

In [ ]:
results_compare_optimizers

Стохастичний градієнтний спуск з learning rate=0.1 показав кращі результати функції помилки. Проте адаптивний алгоритм Adadelta  показала кращі результати в accuracy та найкращий результат в f1. А оскільки ми маємо датасет з дуже несбалансованими класами, дана метрика є найважливішою для оцінки якості роботи моделі.

In [ ]:
def batch_norm_model(epochs, lr):
    
        
    model = Sequential()
    model.add(Input(shape))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(256,
                    activation='tanh',
                    kernel_initializer=initializers.GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Dense(64,
                    activation='tanh',
                    kernel_initializer=initializers.GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Dense(num_targets,
                    activation='softmax',
                    kernel_initializer=initializers.RandomNormal(0, 1, seed=seed)))

    model.compile(
        loss=CategoricalCrossentropy(),
        optimizer=optimizers.Adagrad(learning_rate=lr),
        metrics=metrics)

    history = model.fit(**fit_params,
                        epochs=epochs)
    del model
#     gc.collect()
    
    return history

In [ ]:
results_batch_norm1 = batch_norm_model(epochs=10, lr=0.001)
results_batch_norm2 = batch_norm_model(epochs=10, lr=0.05)

In [ ]:
results_plot(results_batch_norm1, 'batchnorm + lr 0.001')
results_plot(results_batch_norm2, 'batchnorm + lr 0.05')

Процес навчання уповільнився.На тренувальному сеті показник accuracy покращився,проте на валідаційному значно зменшився.Також значення loss та ассuracy значно стабілізувалися. 

In [ ]:
def dropout_model(epochs, p):
    
        
    model = Sequential()
    model.add(Input(shape))
    model.add(Flatten())
    model.add(Dense(256,
                    activation='tanh',
                    kernel_initializer=initializers.GlorotNormal()))
    model.add(Dense(64,
                    activation='tanh',
                    kernel_initializer=initializers.GlorotNormal()))
    model.add(Dropout(p))
    model.add(Dense(num_targets,
                    activation='softmax',
                    kernel_initializer=initializers.RandomNormal(0, 1, seed=seed)))

    model.compile(
        loss=CategoricalCrossentropy(),
        optimizer=optimizers.SGD(learning_rate=0.001),
        metrics=metrics)

    history = model.fit(**fit_params,
                        epochs=epochs)
    del model
#     gc.collect()
    
    return history

In [ ]:
dropout_model1 = dropout_model(epochs=10, p=0.1)
dropout_model0 = dropout_model(epochs=10, p=0.25)
dropout_model2 = dropout_model(epochs=10, p=0.5)

In [ ]:
results_plot(dropout_model0, 'dropout 0.1')
results_plot(dropout_model1, 'dropout 0.25')
results_plot(dropout_model2, 'dropout 0.5')

Показник Dropout вимикає певний відсоток нейронів в залежності від заданого параметру. Чим більше модель перенавчається, тим більший потрібно задавати параметр Dropout.

In [ ]:
def early_stopping_model(epochs):
    
    model = Sequential()
    model.add(Input(shape))
    model.add(Flatten())
    model.add(Dense(256,
                    activation='tanh',
                    kernel_initializer=initializers.GlorotNormal()))
    model.add(Dense(64,
                    activation='tanh',
                    kernel_initializer=initializers.GlorotNormal()))
    model.add(Dense(num_targets,
                    activation='softmax',
                    kernel_initializer=initializers.RandomNormal(0, 1, seed=seed)))
    model.compile(
        loss=CategoricalCrossentropy(),
        optimizer=optimizers.SGD(learning_rate=0.001),
        metrics=metrics)

    history = model.fit(**fit_params,
                        epochs=epochs,
                        callbacks=[
                            EarlyStopping(
                                monitor='val_loss',
                                mode='min',
                                patience=5)
                        ]
                       )
    del model
#     gc.collect()
    
    return history

In [ ]:
early_stopping_res = early_stopping_model(10)

In [ ]:
results_plot(early_stopping_res, 'early stopping model')

In [ ]:
def compare_regularization(regularization_list):
    
    results = pd.DataFrame()
    for regularization in regularization_list:
        
        model = Sequential()
        model.add(Input(shape))
        model.add(Flatten())
        model.add(Dense(256,
                        activation='tanh',
                        kernel_initializer=initializers.GlorotNormal(),
                        kernel_regularizer=regularization,
                        bias_regularizer=regularization))
        model.add(Dense(64,
                        activation='tanh',
                        kernel_initializer=initializers.GlorotNormal(),
                        kernel_regularizer=regularization,
                        bias_regularizer=regularization))
        model.add(Dense(num_targets,
                        activation='softmax',
                        kernel_initializer=initializers.RandomNormal(0, 1, seed=seed),
                        kernel_regularizer=regularization,
                        bias_regularizer=regularization))

        model.compile(
            loss=CategoricalCrossentropy(),
            optimizer=optimizers.SGD(learning_rate=0.001),
            metrics=metrics)
        
        history = model.fit(**fit_params,
                            epochs=10)
        del model
        
        results[regularization] = pd.Series({k:v[-1] for k, v in history.history.items()})
        
    return results

In [ ]:
reg_results = compare_regularization(['l1', 'l2', 'l1_l2'])

In [ ]:
def final_model():
    model = Sequential()
    model.add(Input(shape))
    model.add(Flatten())
    model.add(Dense(256,
                    activation='tanh',
                    kernel_initializer=initializers.GlorotNormal())
                    )
    model.add(Dense(64,
                    activation='tanh',
                    kernel_initializer=initializers.GlorotNormal())
                    )
    model.add(Dense(num_targets,
                    activation='softmax',
                    kernel_initializer=initializers.RandomNormal(0, 1, seed=seed))
                    )

    model.compile(
        loss=CategoricalCrossentropy(),
        optimizer=optimizers.SGD(learning_rate=0.001),
        metrics=metrics)

    return model

In [ ]:
model = final_model()
history = model.fit(**fit_params,
                    epochs=12,
                    callbacks=[EarlyStopping(patience=3)])

In [ ]:
results_plot(history, 'final model')

In [ ]:
ypred = np.argmax(model.predict(X_test),axis=1)
ytest = np.argmax(y_test, axis=1)

cf_matrix = confusion_matrix(ytest, ypred)

plt.figure(figsize=(num_targets+2,num_targets))
ax = sns.heatmap(cf_matrix, annot=True, fmt='g')
plt.show()

In [ ]:
X_new = X_test[11]
noise = np.random.normal(0, .2, X_new.shape)
X_new_noise = X_new + noise
pict = [X_new, X_new_noise]

plt.figure(figsize = (10,10))
colums, row = 1, 2

for i in range(2):  
    plt.subplot(colums, row, i + 1)
    plt.imshow(pict[i], cmap='Greys')
    
plt.show()

In [ ]:
ypred_new = model.predict(X_new.reshape(-1,128,128,3))
ypred_new_val = np.argmax(ypred_new, axis=1)

ypred_new_noise = model.predict(X_new_noise.reshape(-1,128,128,3))
ypred_new_noise_val = np.argmax(ypred_new_noise, axis=1)

print(f'Predicted class w/out noise: {ypred_new_val}, w/ noise {ypred_new_noise_val}')